Copyright 2024 Google, LLC. This software is provided as-is,
without warranty or representation for any use or purpose. Your
use of it is subject to your agreement with Google.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Chat Example

This notebook provides a simple example for interacting with Google's Gemini models for chat using Vertex AI. For more information please visit https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini

Import python libraries

In [1]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig, Part, Tool, ChatSession, FunctionDeclaration
import vertexai.preview.generative_models as generative_models
from vertexai.preview.generative_models import grounding
import requests

Set your project variables. Change "YOUR_PROJECT_ID" to your GCP project ID.

In [2]:
project_id = "YOU_PROJECT_ID"
location = "global"
region = "us-central1"

Instantiate the model

In [3]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    generation_config=GenerationConfig(temperature=0),
)
chat = model.start_chat()

## Start with a simple chat session

Define a simple prompt and send it to the model

In [4]:
prompt = "Why is the sky blue?"

In [5]:
response = chat.send_message(prompt)
#response
print(response.text)

The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown:

* **Sunlight and Colors:** Sunlight, while appearing white, is actually a mixture of all the colors of the rainbow.
* **Earth's Atmosphere:** Our atmosphere is made up of tiny particles like nitrogen and oxygen molecules.
* **Scattering of Light:** When sunlight enters the atmosphere, it collides with these particles. This collision causes the light to scatter in different directions.
* **Blue Light's Advantage:** Blue and violet light have shorter wavelengths compared to other colors in the spectrum. This means they are scattered more effectively by the atmospheric particles.
* **Our Perception:** Our eyes are more sensitive to blue light than violet. As a result, we perceive the sky as blue due to the scattered blue light reaching our eyes from all directions.

**In simpler terms:** Imagine throwing a white ball at a bunch of small objects. The ball represents sunlight, and the objects repre

## Now let's upload some files to the model and ask questions about them

Encode a file into a base64 encoded string
(this file is a public data set from Kaggle - https://www.kaggle.com/datasets/kyanyoga/sample-sales-data)

In [6]:
encoded_file = base64.b64encode(open("sales_data_sample.json", "rb").read()).decode("utf-8")

file_content = Part.from_data(
    data=base64.b64decode(encoded_file), mime_type="text/plain"
)
# supported mime types at https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

Add the file to the model's context window

In [7]:
prompt = "Add this document to your context. If you are able to process it, provide a simple response that it was successfully uploaded"
gemini_results = chat.send_message([file_content, prompt])
print(gemini_results.text)

OK. I've got the sales data. What would you like to do with it? 



In [8]:
prompt = "what's in that document?"
gemini_results = chat.send_message(prompt)
print(gemini_results.text)

The document contains sales data for various food, pet care, and snack products. It includes details like order numbers, quantities, prices, sales dates, customer information, and deal sizes.  

What kind of information are you interested in? I can tell you things like:

* **Total sales for a specific product**
* **Average deal size in a particular country**
* **Number of orders shipped in a given month**

Just let me know what you'd like to explore! 



Encode an image into a base64 encoded string

In [9]:
encoded_file = base64.b64encode(open("stuff_on_a_shelf.jpg", "rb").read()).decode("utf-8")

image_content = Part.from_data(
    data=base64.b64decode(encoded_file), mime_type="image/jpeg"
)
# supported mime types at https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini

Add the image to the model's context window

In [10]:
prompt = "what about this file?"
gemini_results = chat.send_message([image_content, prompt])
print(gemini_results.text)

The file is a picture of a shelf at a store with various cleaning products. The picture shows brands like Pledge, Old English, Method, Weiman, and Resolve. It also shows prices for the products. 



Verify the model has access to both the file and the image

In [11]:
prompt = "OK, how many text and image files have we added to your context window so far?"
gemini_results = chat.send_message(prompt)
print(gemini_results.text)

As a large language model, I don't have a persistent memory or a "context window" in the way you might be thinking. Each time we interact, I process your request and the information you provide anew. 

So, while I can access and process the sales data and the image you provided during our current conversation, I won't remember them once the conversation ends. 

Does that make sense? 



## Let's make this code a little cleaner

Define a function to make the interaction with the model cleaner from a programming perspective 

In [12]:
def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        #role = chunk.candidates[0].content.role)
        text_response.append(chunk.text)
    #print(chat.history)
    return "".join(text_response)

Send some conversational prompts and print the output showing the roles (i.e. like a more traditional chat bot with user: and gemini:)

In [13]:
prompt = "Hello."
print('user: ' + prompt)
print('gemini: ' + get_chat_response(chat, prompt))

user: Hello.
gemini: Hello! 👋 How can I help you today? 😊 



In [14]:
prompt = "Tell me more about that image."
print('user: ' + prompt)
print('gemini: ' + get_chat_response(chat, prompt))

user: Tell me more about that image.
gemini: Please provide me with the image you are referring to! I need to see it to tell you more about it. 😊 

You can describe it, or if you've already shared it in our conversation, just let me know and I can refer back to it. 



In [15]:
prompt = "What are all the colors in a rainbow?"
print('user: ' + prompt)
print(get_chat_response(chat, prompt))

user: What are all the colors in a rainbow?
The colors of a rainbow, in order, are:

* **Red**
* **Orange**
* **Yellow**
* **Green**
* **Blue**
* **Indigo**
* **Violet**

This is often remembered by the acronym **ROYGBIV**. 



In [16]:
prompt = "Why does it appear when it rains?"
print('user: ' + prompt)
print(get_chat_response(chat, prompt))

user: Why does it appear when it rains?
You're right to connect rain and rainbows! Rainbows appear when it rains because:

* **Sunlight:** You need sunlight to create a rainbow.
* **Water Droplets:** Rain provides the water droplets that act like tiny prisms.
* **Refraction and Reflection:** When sunlight enters a water droplet, it bends (refracts) and bounces off the inside (reflects). This separates the white sunlight into its different colors.
* **Angle:**  The angle between the sun, the water droplets, and your eyes needs to be just right (around 42 degrees) for you to see the rainbow.

**In simpler terms:** Imagine sunlight shining through a bunch of tiny, round mirrors (the raindrops). The mirrors bend and bounce the light, splitting it into the colors of the rainbow. You see the rainbow when you're standing in the right spot to catch those colorful reflections. 



## Let's see how to use streaming responses to enhance the interaction and reduce the perceived latency.

Define a new function that will output the respoonse in a stream

In [17]:
def get_chat_response_stream(chat: ChatSession, prompt: str) -> str:
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        print(chunk.text)
        #print(chunk)
        #return chunk.text
    #return "".join(text_response)

In [18]:
prompt = "Hello."
get_chat_response_stream(chat, prompt)

Hello
! 👋  It's nice to hear from you. What can I do
 for you today? 😊 



In [19]:
prompt = "What are all the colors in a rainbow?"
print('user: ' + prompt)
get_chat_response_stream(chat, prompt)

user: What are all the colors in a rainbow?
The
 colors of the rainbow, in order, are:

* **Red**

* **Orange**
* **Yellow**
* **Green**
*
 **Blue**
* **Indigo**
* **Violet**

A helpful acronym to remember the order is **ROYGBIV**. 



In [20]:
prompt = "Why does it appear when it rains?"
get_chat_response_stream(chat, prompt)

Rain
bows are actually not directly caused by rain itself, but rather by the presence of
 water droplets in the air after or during rainfall. Here's how it works
:

* **Sunlight:** Rainbows need sunlight to form.
* **Water Droplets:** The air is filled with water droplets after it rains. These
 droplets act like tiny prisms.
* **Refraction:** When sunlight enters a water droplet, it bends (refracts) as it passes from air to
 water.
* **Dispersion:**  Different colors of light bend at slightly different angles. This process, called dispersion, separates the white sunlight into its spectrum of colors.
* **Reflection:** The light then reflects off the inside surface
 of the water droplet.
* **Second Refraction:** As the light exits the droplet, it refracts again, further separating the colors.
* **Observer's Perspective:** You see a rainbow when you are positioned so that the
 refracted and reflected light from the water droplets reaches your eyes. The angle between the sun, the water dro